In [1]:
# Homework 4
# Center of Mass Position and Velocity
# Sammie Mackie
# built w/in provided jupyter notebook

In [2]:
# import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl

from ReadFile import Read

In [3]:
class CenterOfMass:
# Class to define COM position and velocity properties of a given galaxy 
# and simulation snapshot
    
    
    def __init__(self, filename, ptype):
    # Initialize the instance of this Class with the following properties:
    
        # read data in the given file using Read
        self.time, self.total, self.data = Read(filename)                                                                                             

        #create an array to store indexes of particles of desired Ptype                                
        self.index = np.where(self.data['type'] == ptype)

        # store the mass, positions, velocities of only the particles of the given type
        # the following only gives the example of storing the mass
        self.m = self.data['m'][self.index]
        # write your own code to complete this for positions and velocities
        # the syntax is the same for the other columns of self.data
        self.x = self.data['x'][self.index]
        self.y = self.data['y'][self.index]
        self.z = self.data['z'][self.index]
        self.vx = self.data['vx'][self.index]
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index]


    def COMdefine(self,a,b,c,m):
    # Function to compute the center of mass position or velocity generically
    # input: array (a,b,c) of positions or velocities and the mass
    # returns: 3 floats  (the center of mass coordinates)

        # write your own code to compute the generic COM using Eq. 1 in the homework instructions
        # the centers of mass are just the sum of position * mass devided by the total mass
        # xcomponent Center of mass
        Acom = np.sum(a * m) / np.sum(m)
        # ycomponent Center of mass
        Bcom = np.sum(b * m) / np.sum(m)
        # zcomponent Center of mass
        Ccom = np.sum(c * m) / np.sum(m)
        
        return Acom, Bcom, Ccom
    
    
    def COM_P(self, delta):
    # Function to specifically return the center of mass position and velocity                                         
    # input:                                                                                                           
    #        particle type (1,2,3)                                                                                     
    #        delta (tolerance)                                                                                         
    # returns: One vector, with rows indicating:                                                                                                                                                                            
    #       3D coordinates of the center of mass position (kpc)                                                             

        # Center of Mass Position                                                                                      
        ###########################                                                                                    

        # Try a first guess at the COM position by calling COMdefine                                                   
        XCOM, YCOM, ZCOM = self.COMdefine(self.x, self.y, self.z, self.m)
        # compute the magnitude of the COM position vector.
        # write your own code below
        RCOM = np.sqrt(XCOM**2 + YCOM**2 + ZCOM**2)
        
        # iterative process to determine the center of mass                                                            

        # change reference frame to COM frame                                                                          
        # compute the difference between particle coordinates                                                          
        # and the first guess at COM position
        # write your own code below
        xNew = self.x - XCOM
        yNew = self.y - YCOM
        zNew = self.z - ZCOM
        RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)

        # find the max 3D distance of all particles from the guessed COM                                               
        # will re-start at half that radius (reduced radius)                                                           
        RMAX = max(RNEW)/2.0
        
        # pick an initial value for the change in COM position                                                      
        # between the first guess above and the new one computed from half that volume
        # it should be larger than the input tolerance (delta) initially
        CHANGE = 1000.0

        # start iterative process to determine center of mass position                                                 
        # delta is the tolerance for the difference in the old COM and the new one.    
        
        while (CHANGE > delta):
            # select all particles within the reduced radius (starting from original x,y,z, m)
            # write your own code below (hints, use np.where)
            index2 = np.where(RNEW <= RMAX)
            x2 = self.x[index2]
            y2 = self.y[index2]
            z2 = self.z[index2]
            m2 = self.m[index2]

            # Refined COM position:                                                                                    
            # compute the center of mass position using                                                                
            # the particles in the reduced radius
            # write your own code below
            # same syntax as line 57
            XCOM2, YCOM2, ZCOM2 = self.COMdefine(x2, y2, z2, m2)
            # compute the new 3D COM position
            # write your own code below
            RCOM2 = np.sqrt(XCOM2**2 + YCOM2**2 + ZCOM2**2)

            # determine the difference between the previous center of mass position                                    
            # and the new one.                                                                                         
            CHANGE = np.abs(RCOM - RCOM2)
            # uncomment the following line if you wnat to check this                                                                                               
            # print ("CHANGE = ", CHANGE)                                                                                     

            # Before loop continues, reset : RMAX, particle separations and COM

            # reduce the volume by a factor of 2 again                                                                 
            RMAX = RMAX/2.0
            # check this.                                                                                              
            # print ("maxR", RMAX)                                                                                      

            # Change the frame of reference to the newly computed COM.                                                 
            # subtract the new COM
            # write your own code below
            xNew = self.x - XCOM2
            yNew = self.y - YCOM2
            zNew = self.z - ZCOM2
            RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)

            # set the center of mass positions to the refined values                                                   
            XCOM = XCOM2
            YCOM = YCOM2
            ZCOM = ZCOM2
            RCOM = RCOM2

            # create a vector to store the COM position                                                                                                                                                       
            COMP = [XCOM, YCOM, ZCOM]

        # set the correct units usint astropy and round all values
        # and then return the COM positon vector
        # write your own code below
        return np.around(COMP,2)*u.kpc
        
    

    def COM_V(self, COMX, COMY, COMZ):
        # Center of Mass velocity
        # input: X, Y, Z positions of the COM
        # returns 3D Vector of COM Velocities
        
        # the max distance from the center that we will use to determine the center of mass velocity                   
        RVMAX = 15.0*u.kpc

        # determine the position of all particles relative to the center of mass position
        # write your own code below
        # putting units on the self. parts to get rid of unit error
        xV = self.x*u.kpc - COMX
        yV = self.y*u.kpc - COMY
        zV = self.z*u.kpc - COMZ
        RV = np.sqrt(xV**2 + yV**2 + zV**2)
        
        # determine the index for those particles within the max radius
        # write your own code below
        indexV = np.where(RV < RVMAX)

        # determine the velocity and mass of those particles within the mas radius
        # write your own code below
        vxnew = self.vx[indexV]
        vynew = self.vy[indexV]
        vznew = self.vz[indexV]
        mnew = self.m[indexV]
        
        # compute the center of mass velocity using those particles
        # write your own code below
        VXCOM, VYCOM, VZCOM = self.COMdefine(vxnew, vynew, vznew, mnew)

        # create a vector to store the COM velocity
        # set the correct units usint astropy
        # round all values
        # write your own code below
        COMV = np.around([VXCOM, VYCOM, VZCOM],2) * u.km / u.s

        # return the COM vector                                                                                        
        return COMV

In [4]:
# Create a Center of mass object for the MW, M31 and M33
# below is an example of using the class for MW
MWCOM = CenterOfMass("../../MW_000.txt", 2)

In [5]:
# below gives you an example of calling the class's functions
# MW:   store the position and velocity COM
MW_COMP = MWCOM.COM_P(0.1)
MW_COMV = MWCOM.COM_V(MW_COMP[0], MW_COMP[1], MW_COMP[2])

In [6]:
# now write your own code to answer questions

# making COM object for M31 and calculating its COM position and velocity
M31COM = CenterOfMass("../../M31_000.txt", 2)
M31_COMP = M31COM.COM_P(0.1)
M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])

# making COM object for M33 and calculating its COM position and velocity
M33COM = CenterOfMass("../../M33_000.txt", 2)
M33_COMP = M33COM.COM_P(0.1)
M33_COMV = M33COM.COM_V(M33_COMP[0], M33_COMP[1], M33_COMP[2])

print("MW COM position  = ", MW_COMP)
print("MW COM velocity  = ", MW_COMV)
print("\nM31 COM position = ", M31_COMP)
print("M31 COM velocity = ", M31_COMV)
print("\nM33 COM position = ", M33_COMP)
print("M33 COM velocity = ", M33_COMV)

MW COM position  =  [-2.07  2.95 -1.45] kpc
MW COM velocity  =  [ 0.94  6.32 -1.35] km / s

M31 COM position =  [-377.66  611.43 -284.64] kpc
M31 COM velocity =  [ 72.85 -72.14  49.  ] km / s

M33 COM position =  [-476.22  491.44 -412.4 ] kpc
M33 COM velocity =  [ 44.42 101.78 142.23] km / s


In [7]:
#Calculating the mag. of separation and velocity of the MW and M31
MWtoM31_P = MW_COMP - M31_COMP
magMWtoM31_P = np.around(np.sqrt(MWtoM31_P[0]**2 + MWtoM31_P[1]**2 + MWtoM31_P[2]**2),2)

MWtoM31_V = MW_COMV - M31_COMV
magMWtoM31_V = np.around(np.sqrt(MWtoM31_V[0]**2 + MWtoM31_V[1]**2 + MWtoM31_V[2]**2),2)

print("Distance btwn MW and M31 = ", magMWtoM31_P)
print("Relative velocity of M31 to MW = ", magMWtoM31_V)

#Calculating the mag. of separation and velocity of M31 and M33
M31toM33_P = np.abs(M31_COMP - M33_COMP)
magM31toM33_P = np.around(np.sqrt(M31toM33_P[0]**2 + M31toM33_P[1]**2 + M31toM33_P[2]**2),2)

M31toM33_V = np.abs(M31_COMV - M33_COMV)
magM31toM33_V = np.around(np.sqrt(M31toM33_V[0]**2 + M31toM33_V[1]**2 + M31toM33_V[2]**2),2)

print("\nDistance btwn M31 and M33 = ", magM31toM33_P)
print("Relative velocity of M33 to M31 = ", magM31toM33_V)

Distance btwn MW and M31 =  769.1 kpc
Relative velocity of M31 to MW =  117.74 km / s

Distance btwn M31 and M33 =  201.08 kpc
Relative velocity of M33 to M31 =  199.37 km / s


**Question 1:**

MW COM position  =  [-0.87  2.39 -1.42] kpc

MW COM velocity  =  [-0.47  3.41 -1.33] km / s

M31 COM position =  [-377.66  611.43 -284.64] kpc

M31 COM velocity =  [ 72.85 -72.14  49.  ] km / s

M33 COM position =  [-476.22  491.44 -412.4 ] kpc

M33 COM velocity =  [ 44.42 101.78 142.23] km / s

**Question 2:**

Distance btwn MW and M31 =  770.14 kpc

Relative velocity of M31 to MW =  116.69 km / s


**Question 3:**

Distance btwn M31 and M33 =  201.08 kpc

Relative velocity of M33 to M31 =  199.37 km / s

**Question 4:**

It's important for us to use the iterative process to determine the centers of mass of the galaxies because the precise position and velocity of the COM tells us exactly how the galaxy as a whole will move over a timestep. The COM interacts with every particle, so we need to narrow down exactly where it is at any given time in order to accurately model how it will interact with every particle. We're interested in seeing *how* the MW and M31 will merge, not just the fact that they will merge, so in order to get the details of that event, we need to have a good grasp on how they're moving and also on how their COMs are interacting, which the iterative process gives us by specifying where the COMs are and where they're heading.

**Some comments:** (Feel free to ignore)

Each galaxy's center of mass is at the center of its own coordinate system, so I'm currently not sure how to show M31 and M33 in the MW_000 coordinate system without already knowing their displacement. The subtractions (ex: xNew = X - XCOM) in COM_P cause COM positons to converge towards 0 in x, y and z as they're intended to, I'm just stuck on subtracting the COM displacement (where it actually is) from the original COM calculated in COMdefine

I think I skipped the comment in line 110 *# Before loop continues, reset : RMAX, particle separations and COM* and that's why I'm stuck

*TODO: Need to reset values listed in line 110?*

FIXED